# Collect Data

In [1]:
import numpy as np
from grabscreen import grab_screen
import cv2
import time
from getkeys import key_check
import os

w  = [1,0,0,0,0,0]
a  = [0,1,0,0,0,0]
d  = [0,0,1,0,0,0]
wa = [0,0,0,1,0,0]
wd = [0,0,0,0,1,0]
nk = [0,0,0,0,0,1]

starting_value = 1

while True:
    file_name = 'training_data-{}.npy'.format(starting_value)

    if os.path.isfile(file_name):
        print('File exists, moving along',starting_value)
        starting_value += 1
    else:
        print('File does not exist, starting fresh!',starting_value)
        
        break


def keys_to_output(keys):
    
    output = [0,0,0,0,0,0]

    if 'W' in keys and 'A' in keys:
        output = wa
    elif 'W' in keys and 'D' in keys:
        output = wd
    elif 'W' in keys:
        output = w
    elif 'A' in keys:
        output = a
    elif 'D' in keys:
        output = d
    else:
        output = nk
    return output


def collect(file_name, starting_value):
    file_name = file_name
    starting_value = starting_value
    training_data = []
    for i in list(range(7))[::-1]:
        print(i+1)
        time.sleep(1)

    last_time = time.time()
    paused = False
    print('STARTING!!!')
    while(True):
        
        if not paused:
            screen = grab_screen(region=(0,32,1280,752))
            last_time = time.time()
            screen = cv2.resize(screen, (480,270))
            screen = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
            
            keys = key_check()
            output = keys_to_output(keys)
            training_data.append([screen,output])
            cv2.imshow('window2',screen)            
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

            last_time = time.time()

            if len(training_data) % 1000 == 0:
                print(len(training_data))
                
                if len(training_data) == 5000:
                    np.save(file_name,training_data)
                    print('SAVED')
                    training_data = []
                    starting_value += 1
                    file_name = 'training_data-{}.npy'.format(starting_value)

                    
        keys = key_check()
        if 'T' in keys:
            if paused:
                paused = False
                print('unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                time.sleep(1)

File exists, moving along 1
File exists, moving along 2
File exists, moving along 3
File exists, moving along 4
File does not exist, starting fresh! 5


In [ ]:
collect(file_name, starting_value)

# Train Model

In [ ]:
import numpy as np
from grabscreen import grab_screen
import cv2
import time
import os
import pandas as pd
from tqdm import tqdm
from collections import deque
from models import inception_v3 as googlenet
from random import shuffle


FILE_I_END = 7

WIDTH = 480
HEIGHT = 270
LR = 1e-3
EPOCHS = 200

MODEL_NAME = 'charles_v1'
PREV_MODEL = 'charles_v1'

LOAD_MODEL = True

wl = 0
al = 0
dl = 0

wal = 0
wdl = 0
nkl = 0

w  = [1,0,0,0,0,0]
a  = [0,1,0,0,0,0]
d  = [0,0,1,0,0,0]
wa = [0,0,0,1,0,0]
wd = [0,0,0,0,1,0]
nk = [0,0,0,0,0,1]

model = googlenet(WIDTH, HEIGHT, 3, LR, output=6, model_name=MODEL_NAME)

if LOAD_MODEL:
    model.load(PREV_MODEL)
    print('We have loaded a previous model!!!!')
    



for e in range(EPOCHS):
    data_order = [i for i in range(1,FILE_I_END+1)]
    shuffle(data_order)
    for count,i in enumerate(data_order):
        
        try:
            file_name = 'training_data-{}.npy'.format(i)

            train_data = np.load(file_name)
            print('training_data-{}.npy'.format(i),len(train_data))

            shuffle(train_data)
            train = train_data[:-100]
            test = train_data[-100:]

            X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,3)
            Y = [i[1] for i in train]

            test_x = np.array([i[0] for i in test]).reshape(-1,WIDTH,HEIGHT,3)
            test_y = [i[1] for i in test]

            model.fit({'input': X}, {'targets': Y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
                snapshot_step=500, show_metric=True, run_id=MODEL_NAME)


            if count%1 == 0:
                print('SAVING MODEL!')
                model.save(MODEL_NAME)
                    
        except Exception as e:
            print(str(e))
            
    









#tensorboard --logdir=foo:E:\JLD\Desktop\py_workspace\GTAV\charles\log\charles_v1



# Test Model

In [1]:
import numpy as np
from grabscreen import grab_screen
import cv2
import time
from directkeys import PressKey,ReleaseKey, W, A, S, D
from models import inception_v3 as googlenet
from getkeys import key_check
from collections import deque, Counter
import random
from statistics import mode,mean
import numpy as np
from motion import motion_detection

GAME_WIDTH = 1280
GAME_HEIGHT = 720

how_far_remove = 800
rs = (20,15)
log_len = 25

motion_req = 800
motion_log = deque(maxlen=log_len)

WIDTH = 480
HEIGHT = 270
LR = 1e-3
EPOCHS = 10

choices = deque([], maxlen=5)
hl_hist = 250
choice_hist = deque([], maxlen=hl_hist)

w  = [1,0,0,0,0,0]
a  = [0,1,0,0,0,0]
d  = [0,0,1,0,0,0]
wa = [0,0,0,1,0,0]
wd = [0,0,0,0,1,0]
nk = [0,0,0,0,0,1]

t_time = 0.25

def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    ReleaseKey(S)

def left():
    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    PressKey(A)
    ReleaseKey(S)
    ReleaseKey(D)


def right():
    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)
    
def reverse():
    PressKey(S)
    ReleaseKey(A)
    ReleaseKey(W)
    ReleaseKey(D)


def forward_left():
    PressKey(W)
    PressKey(A)
    ReleaseKey(D)
    ReleaseKey(S)
    
    
def forward_right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(S)

    
def reverse_left():
    PressKey(S)
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)

    
def reverse_right():
    PressKey(S)
    PressKey(D)
    ReleaseKey(W)
    ReleaseKey(A)

def no_keys():

    if random.randrange(0,3) == 1:
        PressKey(W)
    else:
        ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(S)
    ReleaseKey(D)
    


model = googlenet(WIDTH, HEIGHT, 3, LR, output=6)
MODEL_NAME = 'charles_v1'
model.load(MODEL_NAME)

print('We have loaded a previous model!!!!')

def test():
    last_time = time.time()
    for i in list(range(7))[::-1]:
        print(i+1)
        time.sleep(1)

    paused = False
    mode_choice = 0

    screen = grab_screen(region=(0,32,GAME_WIDTH,GAME_HEIGHT+32))
    screen = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
    prev = cv2.resize(screen, (WIDTH,HEIGHT))

    t_minus = prev
    t_now = prev
    t_plus = prev

    while(True):
        
        if not paused:
            screen = grab_screen(region=(0,32,GAME_WIDTH,GAME_HEIGHT+32))
            screen = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)

            last_time = time.time()
            screen = cv2.resize(screen, (WIDTH,HEIGHT))

            delta_count_last = motion_detection(t_minus, t_now, t_plus)

            t_minus = t_now
            t_now = t_plus
            t_plus = screen
            t_plus = cv2.blur(t_plus,(4,4))

            prediction = model.predict([screen.reshape(WIDTH,HEIGHT,3)])[0]
            prediction = np.array(prediction) * np.array([3, 0.1, 0.1, 1.8,  1.8, 0.1])

            mode_choice = np.argmax(prediction)

            if mode_choice == 0:
                straight()
                choice_picked = 'straight'           
            elif mode_choice == 1:
                left()
                choice_picked = 'left'
            elif mode_choice == 2:
                right()
                choice_picked = 'right'
            elif mode_choice == 3:
                forward_left()
                choice_picked = 'forward+left'
            elif mode_choice == 4:
                forward_right()
                choice_picked = 'forward+right'
            elif mode_choice == 5:
                no_keys()
                choice_picked = 'nokeys'

            motion_log.append(delta_count_last)
            motion_avg = round(mean(motion_log),3)
            print('loop took {} seconds. Motion: {}. Choice: {}'.format( round(time.time()-last_time, 3) , motion_avg, choice_picked))
            
            if motion_avg < motion_req and len(motion_log) >= log_len:
                print('WERE PROBABLY STUCK FFS, initiating some evasive maneuvers.')


                quick_choice = random.randrange(0,4)
                
                if quick_choice == 0:
                    reverse()
                    time.sleep(random.uniform(1,2))
                    forward_left()
                    time.sleep(random.uniform(1,2))

                elif quick_choice == 1:
                    reverse()
                    time.sleep(random.uniform(1,2))
                    forward_right()
                    time.sleep(random.uniform(1,2))

                elif quick_choice == 2:
                    reverse_left()
                    time.sleep(random.uniform(1,2))
                    forward_right()
                    time.sleep(random.uniform(1,2))

                elif quick_choice == 3:
                    reverse_right()
                    time.sleep(random.uniform(1,2))
                    forward_left()
                    time.sleep(random.uniform(1,2))

                for i in range(log_len-2):
                    del motion_log[0]
    
        keys = key_check()


        if 'T' in keys:
            if paused:
                paused = False
                time.sleep(1)
            else:
                paused = True
                ReleaseKey(A)
                ReleaseKey(W)
                ReleaseKey(D)
                time.sleep(1)


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
INFO:tensorflow:Restoring parameters from E:\JLD\Desktop\py_workspace\GTAV\charles\charles_v1
We have loaded a previous model!!!!


In [ ]:
test()

7
6
5
4
3
2
1
